In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [ ]:
from shutil import copyfile
os.makedirs('../working/readability/') # create folder
copyfile(src = "../input/package-readability/readability-master/readability-master/readability/__init__.py", dst = "../working/readability/__init__.py")
copyfile(src = "../input/package-readability/readability-master/readability-master/readability/langdata.py", dst = "../working/readability/langdata.py")
copyfile(src = "../input/package-readability/readability-master/readability-master/setup.py", dst = "../working/setup.py")
copyfile(src = "../input/package-readability/readability-master/readability-master/README.rst", dst = "../working/README.rst")

In [ ]:
import readability

# Extract features from excerpt by the readability library

In [ ]:
to_test = pd.read_csv('../input/commonlitreadabilityprize/test.csv')
train = pd.read_csv("../input/commonlitreadabilityprize/train.csv")

In [ ]:
df = pd.DataFrame(columns = ["target"] + ["excerpt"] + 
                  [i for i in readability.getmeasures(to_test.excerpt[0], lang='en')['readability grades']] +
                  [i for i in readability.getmeasures(to_test.excerpt[0], lang='en')["sentence info"]] + 
                 [i for i in readability.getmeasures(to_test.excerpt[0], lang='en')['word usage']] 
                 )
df.target = train.target
df.excerpt = train.excerpt
df_to_test = pd.DataFrame(columns = ["excerpt"] + 
                  [i for i in readability.getmeasures(to_test.excerpt[0], lang='en')['readability grades']] +
                  [i for i in readability.getmeasures(to_test.excerpt[0], lang='en')["sentence info"]] + 
                 [i for i in readability.getmeasures(to_test.excerpt[0], lang='en')['word usage']]  
                 )
df_to_test.excerpt = to_test.excerpt
bl = ["readability grades", "sentence info", "word usage"]
# "readability grades"  train
for i in (df.columns[2:11]):
    for j in range(df.shape[0]):
        df.loc[j, i] = readability.getmeasures(df.excerpt[j], lang='en')[bl[0]][i]
# "readability grades"  test
for i in (df_to_test.columns[1:10]):
    for j in range(df_to_test.shape[0]):
        df_to_test.loc[j, i] = readability.getmeasures(df_to_test.excerpt[j], lang='en')[bl[0]][i]
# 'sentence info'  train
for i in (df.columns[11: 25]):
    for j in range(df.shape[0]):
        df.loc[j, i] = readability.getmeasures(df.excerpt[j], lang='en')[bl[1]][i]
# "sentence info"  test
for i in (df_to_test.columns[10: 24]):
    for j in range(df_to_test.shape[0]):
        df_to_test.loc[j, i] = readability.getmeasures(df_to_test.excerpt[j], lang='en')[bl[1]][i]
# "word usage" train
for i in (df.columns[25: 31]):
    for j in range(df.shape[0]):
        df.loc[j, i] = readability.getmeasures(df.excerpt[j], lang='en')[bl[2]][i]
# "word usage" test
for i in (df_to_test.columns[24: 30]):
    for j in range(df_to_test.shape[0]):
        df_to_test.loc[j, i] = readability.getmeasures(df_to_test.excerpt[j], lang='en')[bl[2]][i]
for i in range(df.shape[0]):
    df.loc[i, "pronoun_b"] =  readability.getmeasures(df.excerpt[[i]])['sentence beginnings']['pronoun']
for i in range(df_to_test.shape[0]):
    df_to_test.loc[i, "pronoun_b"] =  readability.getmeasures(df_to_test.excerpt[[i]])['sentence beginnings']['pronoun']
for i in range(df.shape[0]):
    df.loc[i, "interrogative"] =  readability.getmeasures(df.excerpt[[i]])['sentence beginnings']['interrogative']
for i in range(df_to_test.shape[0]):
    df_to_test.loc[i, "interrogative"] =  readability.getmeasures(df_to_test.excerpt[[i]])['sentence beginnings']['interrogative']
for i in range(df.shape[0]):
    df.loc[i, 'article'] =  readability.getmeasures(df.excerpt[[i]])['sentence beginnings']['article']
for i in range(df_to_test.shape[0]):
    df_to_test.loc[i, 'article'] =  readability.getmeasures(df_to_test.excerpt[[i]])['sentence beginnings']['article']
for i in range(df.shape[0]):
    df.loc[i, "subordination"] =  readability.getmeasures(df.excerpt[[i]])['sentence beginnings']['subordination']
for i in range(df_to_test.shape[0]):
    df_to_test.loc[i, "subordination"] =  readability.getmeasures(df_to_test.excerpt[[i]])['sentence beginnings']['subordination']
for i in range(df.shape[0]):
    df.loc[i, "conjunction_b"] =  readability.getmeasures(df.excerpt[[i]])['sentence beginnings']['conjunction']
for i in range(df_to_test.shape[0]):
    df_to_test.loc[i, "conjunction_b"] =  readability.getmeasures(df_to_test.excerpt[[i]])['sentence beginnings']['conjunction']
for i in range(df.shape[0]):
    df.loc[i, "preposition_b"] =  readability.getmeasures(df.excerpt[[i]])['sentence beginnings']['preposition'] 
for i in range(df_to_test.shape[0]):
    df_to_test.loc[i, "preposition_b"] =  readability.getmeasures(df_to_test.excerpt[[i]])['sentence beginnings']['preposition'] 

In [ ]:
df.head()

In [ ]:
df_to_test.head()

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt
import lightgbm as lgb
from pathlib import Path
import seaborn
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from warnings import simplefilter

In [ ]:
!pip install kaggler

In [ ]:
import kaggler
from kaggler.model import AutoLGB
print(kaggler.__version__)
plt.style.use('fivethirtyeight')
pd.set_option('max_columns', 100)
simplefilter('ignore')

# Concat the training data set and the test data set

In [ ]:
df = pd.concat([df, df_to_test])
df = df.reset_index(drop = True)

# Split the whole data set into balanced training data set and test data set

In [ ]:
# the logic behind this is that, the given training data set 
# and hidden test data set were originally from a whole data set. 
# Staff graded the whole data set first, then split it 
# into training data set and test data set. Now we are 
# inversing the process to see the distribution between 
# training data set and test data set. The split way is
# hard coded right now, so we can use train_test_split
# many times in different seeds to check the average auc 
# score for being more rigorous.

In [ ]:
trn = df.iloc[:1400, ]
tst = df.iloc[1400:, ]
target_col = 'target'
n_fold = 5
seed = 42

In [ ]:
print(trn.shape)
print(tst.shape)

In [ ]:
n_trn = trn.shape[0]
df = pd.concat([trn.drop([target_col,"excerpt"], axis=1), tst.drop("excerpt", axis=1)], axis=0)
print(df.shape)

In [ ]:
df = df.astype(float)
df.head()

In [ ]:
cv = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=seed)
X = df
y = pd.Series(np.concatenate([np.zeros(n_trn,), np.ones(df.shape[0] - n_trn,)]))
p = np.zeros_like(y, dtype=float)
for i, (i_trn, i_val) in enumerate(cv.split(X, y)):
    if i == 0:
        clf = AutoLGB(objective='binary', metric='auc', random_state=seed)
        clf.tune(X.iloc[i_trn], y[i_trn])
        features = clf.features
        params = clf.params
        n_best = clf.n_best
        print(f'{n_best}')
        print(f'{params}')
        print(f'{features}')
    
    trn_data = lgb.Dataset(X.iloc[i_trn], y[i_trn])
    val_data = lgb.Dataset(X.iloc[i_val], y[i_val])
    clf = lgb.train(params, trn_data, n_best, val_data, verbose_eval=100)
    p[i_val] = clf.predict(X.iloc[i_val])
    print(f'CV #{i + 1} AUC: {roc_auc_score(y[i_val], p[i_val]):.6f}')

In [ ]:
print(f'CV AUC: {roc_auc_score(y, p):.6f}')

# Adversarial validation AUC score is far way from 50%. Therefore, we can say that the training and test data sets are not similar in terms of basic feature distributions. In other words, should be careful of a big shake-up at the end of the competition